In [19]:
import pandas as pd
from datetime import datetime

rawpath = "../raw/"

In [24]:
df = pd.read_excel(f"{rawpath}/2022-ES-Tabelle.xlsx", names=['signatur','jahr','datum','datum1111','medienart','sprache','land','koerperschaft_1','koerperschaft_2','person', 'titel','urheber_vorlage','ort','ort_idn','umfang','bemerkung','vorh'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df = df[pd.isna(df.vorh)]


/home/dbsm/.cache/pypoetry/virtualenvs/base-K0tRBB7f-py3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [25]:
def pica_schreiben(row):
    pica = f"""0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 {row.jahr}
"""

    if pd.notna(row.datum):
        d2 = row.datum.split('.')
 
 
    if d2[0] == 'XX':
        if (d2[1]) == '01' or (d2[1]) == '03' or (d2[1]) == '05' or (d2[1]) == '07' or (d2[1]) == '08' or (d2[1]) == '10' or (d2[1]) == '12':
            pica += f"1110 $a01.{d2[1]}.{d2[2]}$b31.{d2[1]}.{d2[2]}$n{row.datum1111}$4ezth\n"
        elif (d2[1]) == '04' or (d2[1]) == '06' or (d2[1]) == '09' or (d2[1]) == '11':
            pica += f"1110 $a01.{d2[1]}.{d2[2]}$b30.{d2[1]}.{d2[2]}$n{row.datum1111}$4ezth\n"
        elif (d2[1]) == '02':
            pica += f"1110 $a01.{d2[1]}.{d2[2]}$b28.{d2[1]}.{d2[2]}$n{row.datum1111}$4ezth\n"
        elif (d2[1]) == 'XX':
            pica += f"1110 $a01.01.{d2[2]}$b31.12.{d2[2]}$n{row.datum1111}$4ezth\n"
    else:
        pica += f"1110 $a{row.datum}$b{row.datum}$n{row.datum1111}$4ezth\n"


    pica += f"""1130 !040445224!
1131 !040082407!
1132 a1-analog;a2-{row.medienart};f1-text;f2-blatt
1500 /1{row.sprache}
"""
    pica += f"1700 /1{'/1'.join(row.land.split(';'))}\n"

    pica += f"3100 {row.koerperschaft_1}$BVerfasser$4aut\n"

    if pd.notna(row.koerperschaft_2):
        pica += f"3110 {row.koerperschaft_2}$BVerfasser$4aut\n"
    
    if pd.notna(row.person):
        pica += f"3010 {row.person}$BVerfasser$4aut\n"
    
    pica += f"""4000 {row.titel} / {row.urheber_vorlage}
4019 Geschäftsrundschreiben$Bobja
4046 {row.ort}
4060 {row.umfang}
4105 !1059146037!
4105 !1077661932!
"""
    if pd.notna(row.bemerkung):
        pica += f"4222 {row.bemerkung}\n"
    
    pica += f"""4700 |BSM|
5560 [RNA]Korrespondenz
5590 [Entstehungsort]
5590 {row.ort_idn}
5591 [Objektgattung]
5591 !042993512!
7001 {now.strftime('%d-%m-%y')} : x
4800 !009033645!
6710 !103243757X!
7100 {row.signatur} @ m
7109 !!DBSM/S!! ; {row.signatur}
\t\n"""
    return pica

In [26]:
now = datetime.now()
for index, row in df.sample(10).iterrows():
    print(pica_schreiben(row))

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1865
1110 $a01.01.1865$b01.01.1865$n1. Januar 1865$4ezth
1130 !040445224!
1131 !040082407!
1132 a1-analog;a2-druck;f1-text;f2-blatt
1500 /1ger
1700 /1XA-DXDE
3100 Heinrich Wehdemann$BVerfasser$4aut
3010 Wehdemann, Heinrich$BVerfasser$4aut
4000 Übergabe der Buchhandlung an den Sohn Carl Wehdemann. N. F.: H. Wehdemann's Buchhandlung. / H. Wehdemann's Buchhandlung
4019 Geschäftsrundschreiben$Bobja
4046 Parchim
4060 4 Exemplare
4105 !1059146037!
4105 !1077661932!
4222 Exemplar a mit eigenhändiger Unterschrift von H. Wehdemann und C. Wehdemann. Empfehlung von W. Einhorn, Leipzig.
4700 |BSM|
5560 [RNA]Korrespondenz
5590 [Entstehungsort]
5590 !042303958!
5591 [Objektgattung]
5591 !042993512!
7001 15-05-23 : x
4800 !009033645!
6710 !103243757X!
7100 Bö-GR/W/335 @ m
7109 !!DBSM/S!! ; Bö-GR/W/335
	

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1886
1110 $a01.07.1886$

In [27]:
now = datetime.now()
with open(f"../dat/rundschreiben-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/rundschreiben-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))

with open(f"../dat/rundschreiben-recent-sample.dat", 'w') as f:
    for index, row in df.sample(5).iterrows():
        f.write(pica_schreiben(row))

In [ ]:
csfn_valnorm -u -c FCV#pica#pica#title -k VALTAB#title -s rundschreiben-recent.stat -e valtest-rundschreiben-recent.err -n -o  BVBIBIDN < rundschreiben-recent.dat.pplus 1>valtest-rundschreiben-recent.out 2>valtest-rundschreiben-recent.stderr
